In [0]:
catalog_name = "rohitb_demo"
schema_name = "pdf_chat"
table_name = "parsed_pdf_docs"
full_table_path = f"{catalog_name}.{schema_name}.{table_name}"

# Structured Output using ai_query

## What are structured outputs?
Structured outputs provide a way to generate structured data in the form of JSON objects from your input data. You can choose to generate text, unstructured JSON objects, and JSON objects that adhere to a specific JSON schema. Structured outputs are supported for chat models served using Foundation Model APIs pay-per-token and provisioned throughput endpoints.

Databricks recommends using structured outputs for the following scenarios:

Extracting data from large amounts of documents. For example, identifying and classifying product review feedback as negative, positive or neutral.
Batch inference tasks that require outputs to be in a specified format.
Data processing, like turning unstructured data into structured data.

In [0]:
response_format = """
{
  "type": "json_schema",
  "json_schema": {
    "name": "Billing Analysis",
    "schema": {
      "properties": {
        "payment_summary": {
          "type": "object",
          "properties": {
            "auto_pay_enrolled": {"type": "boolean"},
            "auto_pay_amount": {"type": "number"},
            "total_due": {"type": "number"},
            "auto_pay_date": {"type": "string", "format": "date"},
            "due_date": {"type": "string", "format": "date"}
          }
        },
        "services": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "company_fees_surcharges": {"type": "number"},
              "phone_number": {"type": "string"},
              "monthly_charge": {"type": "number"},
              "equipment_charge": {"type": ["number", "null"]},
              "government_fees_taxes": {"type": ["number", "null"]},
              "total": {"type": "number"},
              "service_plan": {"type": "string"},
              "user": {"type": "string"}
            }
          }
        },
        "usage_summary": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "phone_number": {"type": "string"},
              "talk_minutes": {"type": "number"},
              "text_messages": {"type": "number"},
              "data_usage_gb": {"type": "number"},
              "user": {"type": "string"}
            }
          }
        },
        "document_metadata": {
          "type": "object",
          "properties": {
            "provider": {"type": "string"},
            "account_number": {"type": "string"},
            "bill_date": {"type": "string", "format": "date"},
            "billing_period": {
              "type": "object",
              "properties": {
                "start_date": {"type": ["string", "null"], "format": "date"},
                "end_date": {"type": ["string", "null"], "format": "date"}
              }
            },
            "account_holder": {"type": "string"}
          }
        },
        "charges_breakdown": {"type": ["object", "null"]},
        "additional_information": {
          "type": "object",
          "properties": {
            "roaming_details": {"type": ["object", "null"]},
            "important_notices": {
              "type": "array",
              "items": {"type": "string"}
            }
          }
        },
        "discounts": {"type": ["object", "null"]}
      },
      "required": [
        "payment_summary",
        "services",
        "usage_summary",
        "document_metadata",
        "charges_breakdown",
        "additional_information",
        "discounts"
      ],
      "title": "Billing Analysis",
      "type": "object"
    },
    "strict": true
  }
}
"""

In [0]:
# Cell 3
query = f"""
        SELECT parsed,
        parse_json(ai_query(
          "databricks-meta-llama-3-1-70b-instruct",
          CONCAT(
            "You are an AI specialized in billing analysis. Review the following comment and provide structured feedback in the given JSON format: ",
          parsed),
          responseFormat => '{response_format}'
        )) AS response
        FROM {full_table_path} limit 10
"""

In [0]:
spark.sql(query).display()

# Key Information Extraction using AI Builder

The Databricks Key Information Extraction (KIE) Private Preview is designed to simplify the process of transforming a large volume of unlabeled text documents into a structured table with extracted information for each document. Examples of KIE we have seen from customers are pulling prices and lease information from contracts, pulling stock trade details from PDFs, pulling important details from news articles, etc. 

Example of using the KIE endpoint in ai_query for batch inference in SQL



This feature leverages automated evaluation capabilities, including MLflow and Agent Evaluation, to enable rapid assessment of the cost-quality tradeoff for your specific extraction task, allowing you to make informed decisions about the balance between accuracy and resource investment. We then provide a fast path to real-time deployment or batch inference. 


In [0]:
%sql 
WITH query_results AS (
  SELECT
    value AS input,
    ai_query(
      'rb-extract-bills',
      input,
      failOnError => false
    ) AS response
  FROM (
    SELECT value 
    FROM read_files('/Volumes/rohitb_demo/pilot_demo/my-volume/pdfs/parsed/', wholeText => true, format => 'text')
    LIMIT 20
  )
)
SELECT
  input,
  try_parse_json(response.result) AS response,
  response.errorMessage AS error
FROM query_results

In [0]:
%sql
create or replace table rohitb_demo.pdf_chat.pdf_structured_output as 
WITH query_results AS (
  SELECT
    value AS input,
    ai_query(
      'rb-extract-bills',
      input,
      failOnError => false
    ) AS response
  FROM (
    SELECT value 
    FROM read_files('/Volumes/rohitb_demo/pilot_demo/my-volume/pdfs/parsed/', wholeText => true, format => 'text')
    LIMIT 20
  )
)
SELECT
  input,
  try_parse_json(response.result) AS response,
  response.errorMessage AS error
FROM query_results